In [58]:
import pandas as pd
import requests
df1 = pd.read_csv(r"C:\Users\MatthewDimaano\Documents\projects\Coursera_Capstone\Toronto_df_lat_long.csv")

In [59]:
df1

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
2,2,M1R,Scarborough,"Maryvale,Wexford",43.750071,-79.295849
3,3,M2H,North York,Hillcrest Village,43.803762,-79.363452
4,4,M4B,East York,"Parkview Hill,Woodbine Gardens",43.706397,-79.309937
5,5,M4G,East York,Leaside,43.709060,-79.363452
6,6,M4M,East Toronto,Studio District,43.659526,-79.340923
7,7,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
8,8,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
9,9,M5V,Downtown Toronto,"Island airport,South Niagara,CN Tower,King and...",43.628947,-79.394420


In [60]:
# select only downtown toronto boroughs
df1=df1[df1.Borough == "Downtown Toronto"]
df1=df1.drop('Unnamed: 0', 1)
df1

,Postcode,Borough,Neighbourhood,Latitude,Longitude
7,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
8,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
9,M5V,Downtown Toronto,"Island airport,South Niagara,CN Tower,King and...",43.628947,-79.394420


In [61]:
CLIENT_ID = '0RAHOZBZPOYYW0LL24PD2QNSYYK0HWVHJLAO4CSPE4QKC5QW' # Foursquare ID



def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [62]:
CLIENT_ID = '0RAHOZBZPOYYW0LL24PD2QNSYYK0HWVHJLAO4CSPE4QKC5QW' # Foursquare ID
CLIENT_SECRET = 'VS0EDRCL5C3ROAQYFYFBINIFMNHE03JEY1Y34XAMR1ZWYFJL' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


In [63]:
toronto_venues = getNearbyVenues(names=df1['Neighbourhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )

Harbourfront,Regent Park
Central Bay Street
Island airport,South Niagara,CN Tower,King and Spadina,Harbourfront West,Bathurst Quay,Railway Lands


In [64]:
print(toronto_venues.shape)

(145, 7)


In [65]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Bay Street,84,84,84,84,84,84
"Harbourfront,Regent Park",48,48,48,48,48,48
"Island airport,South Niagara,CN Tower,King and Spadina,Harbourfront West,Bathurst Quay,Railway Lands",13,13,13,13,13,13


In [66]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 78 uniques categories.


In [67]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Museum,...,Smoothie Shop,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [68]:
toronto_onehot.shape

(145, 79)

In [69]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Museum,...,Smoothie Shop,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,0.011905,...,0.011905,0.023810,0.011905,0.02381,0.011905,0.02381,0.000000,0.011905,0.011905,0.011905
1,"Harbourfront,Regent Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,...,0.000000,0.020833,0.000000,0.00000,0.000000,0.00000,0.041667,0.000000,0.000000,0.020833
2,"Island airport,South Niagara,CN Tower,King and...",0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [70]:
toronto_grouped.shape

(3, 79)

In [71]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1  Italian Restaurant  0.05
2                 Bar  0.04
3                Café  0.04
4        Burger Joint  0.04


----Harbourfront,Regent Park----
         venue  freq
0  Coffee Shop  0.17
1         Park  0.06
2         Café  0.06
3          Pub  0.06
4       Bakery  0.06


----Island airport,South Niagara,CN Tower,King and Spadina,Harbourfront West,Bathurst Quay,Railway Lands----
              venue  freq
0    Airport Lounge  0.15
1   Airport Service  0.15
2  Airport Terminal  0.15
3           Airport  0.08
4     Boat or Ferry  0.08




In [72]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [73]:
import numpy as np

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Bay Street,Coffee Shop,Italian Restaurant,Bar,Sandwich Place,Burger Joint,Bubble Tea Shop,Café,Sushi Restaurant,Spa,Thai Restaurant
1,"Harbourfront,Regent Park",Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Yoga Studio,Farmers Market
2,"Island airport,South Niagara,CN Tower,King and...",Airport Lounge,Airport Service,Airport Terminal,Airport,Sculpture Garden,Harbor / Marina,Boat or Ferry,Plane,Airport Food Court,Airport Gate


In [75]:
from sklearn.cluster import KMeans

In [76]:
# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1])

In [77]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [79]:
toronto_merged = df1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Yoga Studio,Farmers Market
8,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Bar,Sandwich Place,Burger Joint,Bubble Tea Shop,Café,Sushi Restaurant,Spa,Thai Restaurant
9,M5V,Downtown Toronto,"Island airport,South Niagara,CN Tower,King and...",43.628947,-79.394420,1,Airport Lounge,Airport Service,Airport Terminal,Airport,Sculpture Garden,Harbor / Marina,Boat or Ferry,Plane,Airport Food Court,Airport Gate


In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Yoga Studio,Farmers Market
8,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Bar,Sandwich Place,Burger Joint,Bubble Tea Shop,Café,Sushi Restaurant,Spa,Thai Restaurant


In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,1,Airport Lounge,Airport Service,Airport Terminal,Airport,Sculpture Garden,Harbor / Marina,Boat or Ferry,Plane,Airport Food Court,Airport Gate
